In [1]:
%%writefile requirements.txt
sacremoses==0.0.53
sentencepiece==0.1.97
transformers==4.25.1
protobuf~=3.20
torch==1.13.0

Writing requirements.txt


In [ ]:
!pip install -r requirements.txt -q

In [3]:
import os
import json
import torch
import requests
import warnings
import numpy as np

from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

warnings.simplefilter('ignore',UserWarning)
torch.__version__

'1.13.0+cu117'

In [ ]:
def try_model(model_name, text):
    n_min = int(np.median([len(t.split()) for t in text])// 4)
    n_max = int(np.median([len(t.split()) for t in text])// 1.5)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    seq2seq = pipeline("summarization", model=model,tokenizer=tokenizer)
    return seq2seq(text,max_length=n_max, min_length=n_min)

In [6]:
text = """
The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes.
"""
text_split = [s for s in text.splitlines() if s]
try_model("JulesBelveze/t5-small-headline-generator",text_split)

[{'summary_text': 'The Inflation Reduction Act Lowers Drug Costs, Health Care Costs'}]

In [7]:
!git clone https://github.com/datarobot/datarobot-user-models.git

Cloning into 'datarobot-user-models'...


remote: Enumerating objects: 13674, done.
remote: Counting objects: 100% (608/608), done.


remote: Compressing objects: 100% (341/341), done.


remote: Total 13674 (delta 340), reused 467 (delta 248), pack-reused 13066
Receiving objects: 100% (13674/13674), 242.65 MiB | 62.24 MiB/s, done.


Resolving deltas: 100% (9159/9159), done.


Updating files: 100% (725/725), done.


In [8]:
!ls datarobot-user-models/public_dropin_environments/python3_pytorch/

Dockerfile  __init__.py		 env_info.json	requirements.txt
README.md   dr_requirements.txt  fit.sh		start_server.sh


In [9]:
%%writefile datarobot-user-models/public_dropin_environments/python3_pytorch/requirements.txt
sacremoses==0.0.53
sentencepiece==0.1.97
transformers==4.25.1
protobuf~=3.20
torch==1.13.0
numpy==1.22.0
pandas==1.3.0
scikit-learn==0.24.2

Overwriting datarobot-user-models/public_dropin_environments/python3_pytorch/requirements.txt


In [10]:
import datarobot as dr
from datarobot.client import Client

# Connect to DataRobot
Client()


In [ ]:
environment_folder = "./datarobot-user-models/public_dropin_environments/python3_pytorch"
## Create the environment, which will eventually contain versions  ##
execution_environment = dr.ExecutionEnvironment.create(
    name="Python 3 HuggingFace1",
    description="This environment contains hf library.",
)

## Create the environment version ##
environment_version = dr.ExecutionEnvironmentVersion.create(
    execution_environment.id,
    environment_folder,
    max_wait=3600,  # 1 hour timeout
)

[ExecutionEnvironment('[DataRobot] Python 3.9 ONNX Drop-In'),
 ExecutionEnvironment('Python 3 YOLOv5'),
 ExecutionEnvironment('[DataRobot] Julia Drop-In'),
 ExecutionEnvironment('[DataRobot] Legacy Code Environment'),
 ExecutionEnvironment('[DEPRECATED] H2O Drop-In'),
 ExecutionEnvironment('[DataRobot] Python 3.9 PMML Drop-In'),
 ExecutionEnvironment('[DataRobot] R 4.2.1 Drop-In'),
 ExecutionEnvironment('[DataRobot] Python 3.9 PyTorch Drop-In'),
 ExecutionEnvironment('[DataRobot] Java 11 Drop-In (DR Codegen, H2O)'),
 ExecutionEnvironment('[DataRobot] Python 3.9 Scikit-Learn Drop-In'),
 ExecutionEnvironment('[DataRobot] Python 3.9 XGBoost Drop-In'),
 ExecutionEnvironment('[DataRobot] Python 3.9 Keras Drop-In')]

In [ ]:
def save_model(model_name):
    folder_name = model_name.split("/")[1]
    os.mkdir(f"./{folder_name}")
    os.mkdir(f"./{folder_name}/model")
    os.mkdir(f"./{folder_name}/tokenizer")
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model.save_pretrained(f"./{folder_name}/model")
    tokenizer.save_pretrained(f"./{folder_name}/tokenizer")
    return folder_name
folder_name = save_model("JulesBelveze/t5-small-headline-generator")

In [14]:
!ls -l {folder_name}

total 0
drwxr-xr-x 2 nbx nbx  50 Dec 16 14:37 model
drwxr-xr-x 2 nbx nbx 108 Dec 16 14:37 tokenizer


In [15]:
%%writefile {folder_name}/custom.py
import time
import json
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM


def load_model(dummy):

    tokenizer1 = AutoTokenizer.from_pretrained("./tokenizer")
    model1 = AutoModelForSeq2SeqLM.from_pretrained("./model")
    seq2seq1 = pipeline("summarization", model=model1, tokenizer=tokenizer1)

    return seq2seq1


def score_unstructured(model, data, query, **kwargs):
    # Handle incoming data
    if not data:
        return data
    if isinstance(data, bytes):
        data = data.decode("utf8")

    time_before_model = time.perf_counter()

    data = [s for s in data.splitlines() if s]
    prediction = model(data)

    time_after_model = time.perf_counter()

    # Structure json output
    output_dict = {
        "prediction": prediction,
        "model_run_time_seconds": time_after_model - time_before_model,
    }

    # Serialized output
    serialized_output = json.dumps(output_dict)
    return serialized_output


if __name__ == "__main__":
    test = """Videos that say approved vaccines are dangerous and cause autism, cancer or infertility are among those that will be taken down, the company said.  The policy includes the termination of accounts of anti-vaccine influencers.  Tech giants have been criticised for not doing more to counter false health information on their sites.  In July, US President Joe Biden said social media platforms were largely responsible for people's scepticism in getting vaccinated by spreading misinformation, and appealed for them to address the issue.  YouTube, which is owned by Google, said 130,000 videos were removed from its platform since last year, when it implemented a ban on content spreading misinformation about Covid vaccines.  In a blog post, the company said it had seen false claims about Covid jabs "spill over into misinformation about vaccines in general". The new policy covers long-approved vaccines, such as those against measles or hepatitis B.  "We're expanding our medical misinformation policies on YouTube with new guidelines on currently administered vaccines that are approved and confirmed to be safe and effective by local health authorities and the WHO," the post said, referring to the World Health Organization."""
    model = load_model(None)
    preds = score_unstructured(model=model, data=test, query=None)
    print(preds)


Writing t5-small-headline-generator/custom.py


In [16]:
%cd {folder_name}

/nbx/t5-small-headline-generator


In [17]:
!python custom.py

{"prediction": [{"summary_text": "YouTube says it's seen false claims about Covid jabs \"spill over into misinformation\" 'We're expanding our medical misinformation policies on YouTube,' it says"}], "model_run_time_seconds": 0.6249331739963964}


In [18]:
%cd ..

/nbx


In [ ]:
custom_model_folder = f"/nbx/{folder_name}"
## Create the custom model ##
custom_model = dr.CustomInferenceModel.create(
    name=folder_name,
    target_type=dr.TARGET_TYPE.UNSTRUCTURED,
    language='python',
    maximum_memory=4294967296
)

## Create the custom model version ##
model_version = dr.CustomModelVersion.create_clean(
    custom_model_id=custom_model.id,
    folder_path=custom_model_folder,
    base_environment_id=execution_environment.id
)

In [ ]:
custom_model_label = folder_name
prediction_server = dr.PredictionServer.list()[0]
deployment = dr.Deployment.create_from_custom_model_version(
    model_version.id,
    label=custom_model_label,
    default_prediction_server_id=prediction_server.id,
    max_wait=3600,  # 1 hour timeout
)

[PredictionServer(https://mlops.dynamic.orm.datarobot.com),
 PredictionServer(https://datarobot-cfds.dynamic.orm.datarobot.com),
 PredictionServer(https://cfds-ccm-prod.orm.datarobot.com)]

In [21]:
# Make predictions on the custom model deployment
test = """Videos that say approved vaccines are dangerous and cause autism, cancer or infertility are among those that will be taken down, the company said.  
The policy includes the termination of accounts of anti-vaccine influencers.  Tech giants have been criticised for not doing more to counter false health information on their sites.  In July, US President Joe Biden said social media platforms were largely responsible for people's scepticism in getting vaccinated by spreading misinformation, and appealed for them to address the issue.  YouTube, which is owned by Google, said 130,000 videos were removed from its platform since last year, when it implemented a ban on content spreading misinformation about Covid vaccines.  In a blog post, the company said it had seen false claims about Covid jabs "spill over into misinformation about vaccines in general". The new policy covers long-approved vaccines, such as those against measles or hepatitis B.  "We're expanding our medical misinformation policies on YouTube with new guidelines on currently administered vaccines that are approved and confirmed to be safe and effective by local health authorities and the WHO," the post said, referring to the World Health Organization."""
url = '{}/predApi/v1.0/deployments/{}/predictionsUnstructured'.format(prediction_server.url, deployment.id)
headers = dr.client.get_client().headers
headers['datarobot-key'] = '544ec55f-61bf-f6ee-0caf-15c7f919a45d'
headers['Content-Type'] = 'text/plain;UTF-8'

response = requests.post(url, headers=headers, data=test)

predictions = response.json()
print(predictions)

{'prediction': [{'summary_text': 'The company says approved vaccines are dangerous and cause autism, cancer or infertility, according to a video released by the company .'}, {'summary_text': 'YouTube says it saw false claims about Covid jabs "spill over into misinformation" \'We\'re expanding our medical misinformation policies on YouTube,\' blog post says'}], 'model_run_time_seconds': 4.530242481967434}
